# SSH Setup and Remote Development Lab

**Estimated Time:** 30-45 minutes 
**Prerequisites:** SSH client installed, access to a remote server (optional)

---

## Learning Objectives

By completing this lab, you will:
- ✅ Generate and manage SSH keys
- ✅ Configure SSH for convenient access
- ✅ Set up VS Code Remote-SSH
- ✅ Use SSH port forwarding
- ✅ Implement security best practices

---

## Exercise 1: Verify SSH Installation

Let's verify that SSH is installed and check the SSH directory structure.

### Check SSH Version

First, verify that the SSH client is installed:

In [ ]:
# Check SSH version
!ssh -V

### Examine SSH Directory

Check if the `.ssh` directory exists and view its contents:

In [ ]:
# Check SSH directory
import os

ssh_dir = os.path.expanduser('~/.ssh')
print(f"SSH directory: {ssh_dir}")
print(f"Exists: {os.path.exists(ssh_dir)}")

In [ ]:
# List SSH directory contents (if exists)
!ls -la ~/.ssh 2>/dev/null || echo 'Directory ~/.ssh does not exist yet'

---

## Exercise 2: Check for Existing SSH Keys

Before generating new keys, let's check if you already have SSH keys.

In [ ]:
# Check for existing keys
import os

key_types = ['id_rsa', 'id_ed25519', 'id_ecdsa']
existing_keys = []

for key_type in key_types:
    key_path = os.path.expanduser(f'~/.ssh/{key_type}')
    if os.path.exists(key_path):
        existing_keys.append(key_type)
        print(f"✅ Found: {key_type}")

if not existing_keys:
    print("❌ No SSH keys found.")
else:
    print(f"\n⚠️  Existing keys: {', '.join(existing_keys)}")

**Note:** If you have existing keys, consider using them or backing them up before generating new ones.

---

## Exercise 3: Generate SSH Keys

SSH keys provide secure authentication without passwords. 

We'll use Ed25519, the modern and recommended algorithm.

### Why Ed25519?

- **Smaller keys**: 256 bits (vs 2048-4096 for RSA)
- **Faster**: Much quicker key generation and signing
- **More secure**: Modern elliptic curve cryptography
- **Widely supported**: Works with GitHub, GitLab, servers

### Generate Ed25519 Key

Run this command in your **terminal** (not in this notebook):

```bash
ssh-keygen -t ed25519 -C "your_email@example.com"
```

**Follow the prompts:**

1. Press Enter to accept default location (`~/.ssh/id_ed25519`)
2. Enter a strong passphrase (recommended for security)
3. Confirm the passphrase

**Why we don't run it here:** Automatically generating keys in a notebook could overwrite existing keys or create keys without proper user interaction.

### View Your Public Key

After generating your key, you can view the public key (safe to share):

In [ ]:
# View public key
import os

pub_key_path = os.path.expanduser('~/.ssh/id_ed25519.pub')

try:
    with open(pub_key_path, 'r') as f:
        public_key = f.read()
    print("Your Ed25519 public key:")
    print("=" * 70)
    print(public_key)
    print("=" * 70)
except FileNotFoundError:
    print("❌ Public key not found at:", pub_key_path)
    print("Generate it first using: ssh-keygen -t ed25519")

---

## Exercise 4: SSH Configuration File

The SSH config file (`~/.ssh/config`) lets you create shortcuts for your SSH connections.

### Understanding SSH Config Structure

The SSH config file uses a simple structure:
- Each host configuration starts with `Host <name>`
- Configuration options are indented under each `Host` block
- Options apply to all matching hosts until the next `Host` block

Here's an example configuration file with different connection scenarios:

### Example SSH Configuration

Here's a complete example of an SSH config file:

```bash
# SSH Configuration File
# Location: ~/.ssh/config
# Default settings for all hosts

Host *
    ServerAliveInterval 60
    ServerAliveCountMax 3

# Example: Personal server
Host myserver
    HostName 192.168.1.100
    User username
    Port 22
    IdentityFile ~/.ssh/id_ed25519

# Example: Cloud server
Host cloud-gpu
    HostName cloud.example.com
    User ubuntu
    Port 22
    IdentityFile ~/.ssh/id_ed25519
    ForwardAgent yes

# Example: Jump host (bastion)
Host internal-server
    HostName 10.0.0.50
    User admin
    ProxyJump myserver
```

### Benefits of SSH Config

With the above configuration, instead of typing:

```bash
ssh -i ~/.ssh/id_ed25519 username@192.168.1.100
```

You can simply type:

```bash
ssh myserver
```

### Check Your Current Config

You can verify if you have an SSH config file:

```bash
# Check if config file exists
ls -la ~/.ssh/config

# View your current config (if it exists)
cat ~/.ssh/config
```

**Note:** If the file doesn't exist, you can create it with:
```bash
touch ~/.ssh/config
chmod 600 ~/.ssh/config
```

---

## Exercise 5: SSH Agent

SSH agent manages your private keys and remembers your passphrase for the duration of your session.

### Check if SSH Agent is Running

In [ ]:
# Check for SSH agent process
!pgrep -l ssh-agent || echo "SSH agent is not currently running"

### Starting SSH Agent and Adding Keys

Run these commands in your **terminal**:

```bash
# Start SSH agent (if not running)
eval "$(ssh-agent -s)"
# Add your SSH key
ssh-add ~/.ssh/id_ed25519
# List added keys
ssh-add -l
```

**What this does:**
- `ssh-agent` runs in the background and stores your decrypted private key
- `ssh-add` adds your key to the agent (prompts for passphrase once)
- For the rest of your session, you won't need to enter the passphrase

### Verify Keys in Agent

In [ ]:
# List keys currently managed by SSH agent
!ssh-add -l 2>/dev/null || echo "No keys in agent or agent not running"

---

## Exercise 6: Test SSH Connection

If you have access to a remote server, test your SSH connection.

### Test with Verbose Output

Use verbose mode to see detailed connection information:

```bash
ssh -vvv user@hostname
```

Verbose flags:
- `-v`: Verbose (basic debug info)
- `-vv`: More verbose
- `-vvv`: Maximum verbosity (full debug)

### Test GitHub SSH Connection

GitHub provides a test endpoint to verify your SSH setup:

In [ ]:
# Test GitHub SSH (safe to run, just tests connection)
!ssh -T git@github.com 2>&1 || echo "\nNote: GitHub denies shell access by design"

**Expected result:** You'll get a message like "Hi username! You've successfully authenticated" even though it says "Permission denied" - this is normal. GitHub doesn't allow shell access, only git operations.

---

## Exercise 7: SSH Port Forwarding

Port forwarding lets you securely access remote services through SSH tunnels.

### Types of Port Forwarding

**1. Local Port Forwarding** (access remote service locally)

Forward remote Jupyter server to local port:

```bash
ssh -L 8889:localhost:8888 user@remote-server
```

Then access in browser: `http://localhost:8889`

**2. Remote Port Forwarding** (expose local service remotely)

Make local development server accessible from remote:

```bash
ssh -R 8000:localhost:3000 user@remote-server
```

**3. Dynamic Port Forwarding** (SOCKS proxy)

Create a SOCKS proxy:

```bash
ssh -D 9090 user@remote-server
```

**4. Background Tunneling**

Run SSH tunnel in background:

```bash
ssh -fN -L 8889:localhost:8888 user@remote-server
```

- `-f`: Go to background
- `-N`: Don't execute remote command

### Common Use Cases

- **Jupyter notebooks**: Access remote computation locally
- **Databases**: Connect to remote MySQL/PostgreSQL securely
- **Web development**: Test local sites from remote locations
- **Bypassing firewalls**: Access services through allowed SSH port

---

## Exercise 8: VS Code Remote-SSH Setup

VS Code's Remote-SSH extension enables seamless remote development.

### Setup Steps

**1. Install Extension**
- Open VS Code Extensions (Ctrl+Shift+X)
- Search for "Remote - SSH"
- Install by Microsoft

**2. Configure SSH Host**
- Press F1 or Ctrl+Shift+P
- Type: "Remote-SSH: Open Configuration File"
- Select your SSH config file (`~/.ssh/config`)
- Add your server configuration

**3. Connect to Server**
- Press F1 or Ctrl+Shift+P 
- Type: "Remote-SSH: Connect to Host"
- Select your configured host
- VS Code will install server components (first time)

**4. Verify Connection**
- Terminal opens on remote machine
- File explorer shows remote files
- Extensions can be installed on remote

### VS Code Handles

- ✅ Automatic port forwarding (Jupyter, web servers)
- ✅ File editing with local performance
- ✅ Integrated terminal on remote
- ✅ Extension syncing to remote
- ✅ Git operations on remote code

---

## Exercise 9: Security Best Practices

Let's verify your SSH setup follows security best practices.

### Example

```python
print("=" * 70)
print("SSH Security Permissions Check")
print("=" * 70)
```

### Security Checklist

**File Permissions:**
- ✅ `~/.ssh` directory: `700` (drwx------)
- ✅ Private keys: `600` (-rw-------)
- ✅ Public keys: `644` (-rw-r--r--)
- ✅ Config file: `600` (-rw-------)
- ✅ `authorized_keys`: `600` (-rw-------)

**Key Management:**
- ✅ Use Ed25519 (modern and secure)
- ✅ Protect private keys with passphrases
- ✅ Never share private keys
- ✅ Use SSH agent for convenience
- ✅ Rotate keys periodically

**Connection Security:**
- ✅ Disable password authentication on servers
- ✅ Use SSH keys exclusively
- ✅ Keep software updated
- ✅ Use `ServerAliveInterval` to prevent timeouts

---

## Exercise 10: Common SSH Issues and Solutions

Here are the most common SSH problems and how to fix them.

### 1. Permission Denied (Public Key)

**Symptoms:**
```
Permission denied (publickey).
```

**Possible causes:**
- Public key not added to remote `~/.ssh/authorized_keys`
- Wrong private key being used
- File permissions incorrect

**Solutions:**
```bash
# Add your public key to remote server
ssh-copy-id user@host
# Or manually:
cat ~/.ssh/id_ed25519.pub | ssh user@host 'mkdir -p ~/.ssh && cat >> ~/.ssh/authorized_keys'
# Check permissions on remote
ssh user@host 'chmod 700 ~/.ssh && chmod 600 ~/.ssh/authorized_keys'
# Specify key explicitly
ssh -i ~/.ssh/id_ed25519 user@host
```

### 2. Connection Timeout

**Symptoms:**
```
ssh: connect to host example.com port 22: Connection timed out
```

**Possible causes:**
- Wrong hostname or IP
- Firewall blocking port 22
- Server not running SSH service

**Solutions:**
```bash
# Test network connectivity
ping example.com
# Check if port 22 is accessible
nc -zv example.com 22
# Try alternative port if server uses non-standard
ssh -p 2222 user@host
```

### 3. Host Key Verification Failed

**Symptoms:**
```
WARNING: REMOTE HOST IDENTIFICATION HAS CHANGED!
```

**Cause:** Remote server's host key changed (new server, reinstallation)

**Solution:**
```bash
# Remove old host key (only if you trust the change!)
ssh-keygen -R hostname
# Or edit ~/.ssh/known_hosts manually
```

**⚠️ Warning:** Only remove host keys if you're certain the server was legitimately changed!

### 4. SSH Agent Not Working

**Symptoms:** Keep being prompted for passphrase

**Solutions:**
```bash
# Start SSH agent
eval "$(ssh-agent -s)"
# Add your key
ssh-add ~/.ssh/id_ed25519
# Verify keys are loaded
ssh-add -l
# Check agent environment variables
echo $SSH_AUTH_SOCK
echo $SSH_AGENT_PID
```

### 5. Too Many Authentication Failures

**Symptoms:**
```
Received disconnect: Too many authentication failures
```

**Cause:** SSH tries too many keys

**Solution:**
```bash
# Specify exact key to use
ssh -i ~/.ssh/id_ed25519 user@host
# Or add to ~/.ssh/config:
Host myserver
    IdentitiesOnly yes
    IdentityFile ~/.ssh/id_ed25519
```

### 6. Connection Drops Frequently

**Cause:** Idle timeout

**Solution:** Add to `~/.ssh/config`:
```
Host *
    ServerAliveInterval 60
    ServerAliveCountMax 3
```

---

## Summary

### Commands Learned

**Key Management:**
```bash
ssh-keygen -t ed25519 -C "email@example.com" # Generate key
ssh-add ~/.ssh/id_ed25519 # Add to agent
ssh-add -l # List keys in agent
ssh-copy-id user@host # Copy public key
```

**Connections:**
```bash
ssh user@host # Basic connection
ssh -i ~/.ssh/key user@host # Specify key
ssh -p 2222 user@host # Different port
ssh -v user@host # Verbose output
```

**Port Forwarding:**
```bash
ssh -L local:remote:port user@host # Local forward
ssh -R remote:local:port user@host # Remote forward
ssh -D port user@host # Dynamic (SOCKS)
ssh -fN -L port:localhost:port user@host # Background
```

**File Operations:**
```bash
scp file user@host:/path # Copy file to remote
scp user@host:/path/file . # Copy file from remote
rsync -avz folder/ user@host:/path # Sync folder
```

### Skills Acquired

✅ Generate and manage SSH keys (Ed25519)
✅ Configure SSH for convenience (`~/.ssh/config`)
✅ Use SSH agent for password-less authentication
✅ Set up port forwarding for remote services
✅ Configure VS Code Remote-SSH
✅ Verify and fix file permissions
✅ Troubleshoot common SSH issues
✅ Test connections with GitHub

### Next Steps

- Set up SSH keys on GitHub/GitLab
- Configure remote development environment
- Practice port forwarding with Jupyter
- Explore SSH tunneling for databases
- Learn about SSH certificates (advanced)